In [26]:
import sys
import os
import csv
import pandas as pd
import numpy
import config
import spacy
from spacy_cld import LanguageDetector

In [27]:
print(config)

<module 'config' from '/Users/daniel/Github/MUITSS-ASPY/config.py'>


In [28]:
PROJECT_PATH = config.data_directory

In [29]:
df = pd.read_csv('{}/{}.csv'.format(PROJECT_PATH, 'songs'))

In [30]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,82,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,day,12
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,71,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,day,67
2,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,72,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,year,12
3,6m59VvDUi0UQsB2eZ9wVbH,Poison,68,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,day,10
4,63vL5oxWrlvaJ0ayNaQnbX,Istanbul,74,153813,False,They Might Be Giants,6zB02lwP6L6ZH32nggQiJT,Flood,7FwAtuhhWivxvK4aPgyyUD,1990-01-02,day,19


In [40]:
pattern = r'[^a-z0-9A-z]'
df['p_name'] = df['name'].str.replace(pattern, ' ').str.lower()
df['p_artist'] = df['artist'].str.replace(pattern, ' ').str.lower()
df['p_artist'] = df['artist'].str.replace(pattern, ' ').str.lower()
df['file_name'] = df.apply(lambda x: config.data_directory + '/' +x['p_artist'].replace(' ', '').lower()+'-'+x['p_name'].replace(' ', '').lower()+'.txt'
, axis=1)

df['has_lyrics'] = 0

In [42]:
for index, row in df.iterrows():
    exists = os.path.isfile(row['file_name'])
    if exists:
        df.loc[index, 'has_lyrics'] = exists

In [43]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks,p_name,p_artist,has_lyrics,file_name
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,82,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,day,12,thunderstruck,ac dc,True,./_data/acdc-thunderstruck.txt
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,71,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,day,67,don t go breaking my heart,elton john,True,./_data/eltonjohn-dontgobreakingmyheart.txt
2,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,72,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,year,12,man in the box,alice in chains,True,./_data/aliceinchains-maninthebox.txt
3,6m59VvDUi0UQsB2eZ9wVbH,Poison,68,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,day,10,poison,bell biv devoe,True,./_data/bellbivdevoe-poison.txt
4,63vL5oxWrlvaJ0ayNaQnbX,Istanbul,74,153813,False,They Might Be Giants,6zB02lwP6L6ZH32nggQiJT,Flood,7FwAtuhhWivxvK4aPgyyUD,1990-01-02,day,19,istanbul,they might be giants,True,./_data/theymightbegiants-istanbul.txt


In [34]:
df.groupby('has_lyrics').count()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks,p_name,p_artist
has_lyrics,,,,,,,,,,,,,,
0,12,12,12,12,12,12,12,12,12,12,12,12,12,12
True,38,38,38,38,38,38,38,38,38,38,38,38,38,38


In [35]:
nlp = spacy.load('en')

In [36]:
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)
doc = nlp('This is some English text.')

doc._.languages
doc._.language_scores['en']

0.96

In [37]:
def isTextInEnglish(text):
    doc = nlp(text)
    languages = doc._.languages
    try:
        score = doc._.language_scores['en']
        return score > 0.75
    except:
        return False

In [38]:
english_text = 'Wild beast I"ll make you mine Taste your kiss Sweet lips divine She got moves, drips of sweat She got eyes of blue, body curves, legs that"ll break your nerves Fire, when she"s going down Fire, then she make you drown Fire, then she blow you round Yeah, you want some fun Fire your guns, Fire your guns, Fire your guns, Fire your guns  That lady bites Hard style Drips of sex Eyes are wild She got moves Long long legs She got eyes of blue, body curves, Legs that"ll shake the earth Fire, when she"s going down Fire, then she make you drown Fire, then she blow you round Yeah, you want some fun Fire your guns, Fire your guns, Fire your guns, Fire your guns  Fire, when she"s going down Fire, then she make you drown Fire, then she blow you round Yeah, you want some fun Fire your guns, Fire your guns, Fire your guns, Fire your guns Fire your guns, Fire your guns, Fire your guns, Fire your guns Fire your guns Fire your guns '
spanish_text = 'La luna es una naranja del color del carmin a media noche Descalza tras la ventana viendo el madrigal ir golfo y canalla Las motos el rock and roll cuero tachuelas y alcohol que estas haciendo ahora yo intento que el corazon que es un gran muelle de acero no se me salga del pecho Te necesito ya ves odio reconocer que necesito tener tu aliento para estar bien que no hay mejor alimento que el que me das con tus besos Llama por favor soy un cero a la izquierda soy un globo sin gas un barco en alta mar sin patron a la deriva Llama por favor estoy tan indefensa me falta voluntad no me puedo concentrar y no encuentro la salida, no La radio con sus propuestas para un mundo feliz me desconcierta Me pasan por la cabeza mil locuras que no tomo ni en cuenta y aunque me duele esta ves tengo que reconocer que necesito tener tu cuerpo para estar bien enamorada, perdida mi mente te necesita [se repite] '

In [39]:
doc_english_text = nlp(english_text)
doc_spanish_text = nlp(spanish_text)
print(doc_english_text._.languages) 
print(doc_english_text._.language_scores['en'])  
print(doc_spanish_text._.languages)
print('english_text '+ str(isTextInEnglish(english_text)))
print('spanish_text '+ str(isTextInEnglish(spanish_text)))


['en']
0.99
['es']
english_text True
spanish_text False


In [58]:
def read_file(file):
    text_file = open('{}'.format(file), "r")
    text = text_file.read()
    text_file.close()

    return text

def clean_text(text):
    return text.replace('"',"'").replace('“','"').replace('”','"').replace('’','"').replace('‘','"')

def add_lyrics(row):
    if row['has_lyrics']:
        return clean_text(read_file(row['file_name']))
    else:
        return ''

In [61]:
df['lyrics'] = df.apply(lambda x: add_lyrics(x), axis=1)
df['is_english'] = df.apply(lambda x: isTextInEnglish(x['lyrics']), axis=1)


In [63]:
df.head(30)

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks,p_name,p_artist,has_lyrics,file_name,lyrics,is_english
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,82,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,day,12,thunderstruck,ac dc,True,./_data/acdc-thunderstruck.txt,Thunder [x10] I was caught In the middle of ...,True
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,71,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,day,67,don t go breaking my heart,elton john,True,./_data/eltonjohn-dontgobreakingmyheart.txt,Don't go breaking my heart I couldn't if I t...,True
2,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,72,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,year,12,man in the box,alice in chains,True,./_data/aliceinchains-maninthebox.txt,I'm the man in the box [Album version:] Buri...,True
3,6m59VvDUi0UQsB2eZ9wVbH,Poison,68,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,day,10,poison,bell biv devoe,True,./_data/bellbivdevoe-poison.txt,"[Michael Bivins:] Yeah, Spiderman and Freeze...",True
4,63vL5oxWrlvaJ0ayNaQnbX,Istanbul,74,153813,False,They Might Be Giants,6zB02lwP6L6ZH32nggQiJT,Flood,7FwAtuhhWivxvK4aPgyyUD,1990-01-02,day,19,istanbul,they might be giants,True,./_data/theymightbegiants-istanbul.txt,Istanbul was Constantinople Now it's Istanbu...,True
5,3Ti0GdlrotgwsAVBBugv0I,Can I Kick It?,71,251573,False,A Tribe Called Quest,09hVIj6vWgoCDtT03h8ZCa,People's Instinctive Travels and the Paths of ...,3kV0i1qqudjf0PGawJ4jck,1990-04-17,day,17,can i kick it,a tribe called quest,True,./_data/atribecalledquest-canikickit.txt,"[Q-Tip] Can I kick it? (Yes, you can!) [7X]...",True
6,1gVgkQFOKa8Wc1HYsJtPdH,More Than Words,70,334106,False,Extreme,6w7j5wQ5AI5OQYlcM15s2L,Extreme II - Pornograffitti,7DKHQxJTI32UyCdDdGwvRC,1990-01-01,day,13,more than words,extreme,True,./_data/extreme-morethanwords.txt,Saying 'I love you' Is not the words I want ...,True
7,6NdcSEhpGGAYXNnnhGS2s6,What I Like About You,69,175400,False,The Romantics,3daM7asS0gCFvyLemNx2EE,What I Like About You (And Other Romantic Hits),5ZwUOFZdWQ81RYMwXc4j3B,1990,year,10,what i like about you,the romantics,True,./_data/theromantics-whatilikeaboutyou.txt,Hey Uh huh huh Hey Uh huh huh What I like a...,True
8,2SgbR6ttzoNlCRGQOKjrop,Cowboys From Hell,71,243533,True,Pantera,14pVkFUHDL207LzLHtSA18,Cowboys From Hell,5szY4sBOSD6IFjFN1RtWTe,1990-07-20,day,12,cowboys from hell,pantera,True,./_data/pantera-cowboysfromhell.txt,Under the lights where we stand tall Nobody ...,True
9,1B75hgRqe7A4fwee3g3Wmu,U Can't Touch This,73,257360,False,MC Hammer,2rblp9fJo16ZPTcKDtlmKW,Please Hammer Don't Hurt 'Em,4r1WecJyt5FOhglysp9zhN,1990-02-20,day,13,u can t touch this,mc hammer,True,./_data/mchammer-ucanttouchthis.txt,"You can't touch this [5x] My, my, my, my mu...",True
